In [1]:
import pandas as pd

# Load your dataset
data_file = "cleaned_and_merged_crimedata.csv"
df = pd.read_csv(data_file)

# Dictionary to map state abbreviations to full names
state_abbreviations = {
    "AL": "Alabama", "AK": "Alaska", "AZ": "Arizona", "AR": "Arkansas",
    "CA": "California", "CO": "Colorado", "CT": "Connecticut", "DE": "Delaware",
    "FL": "Florida", "GA": "Georgia", "HI": "Hawaii", "ID": "Idaho",
    "IL": "Illinois", "IN": "Indiana", "IA": "Iowa", "KS": "Kansas",
    "KY": "Kentucky", "LA": "Louisiana", "ME": "Maine", "MD": "Maryland",
    "MA": "Massachusetts", "MI": "Michigan", "MN": "Minnesota", "MS": "Mississippi",
    "MO": "Missouri", "MT": "Montana", "NE": "Nebraska", "NV": "Nevada",
    "NH": "New Hampshire", "NJ": "New Jersey", "NM": "New Mexico", "NY": "New York",
    "NC": "North Carolina", "ND": "North Dakota", "OH": "Ohio", "OK": "Oklahoma",
    "OR": "Oregon", "PA": "Pennsylvania", "RI": "Rhode Island", "SC": "South Carolina",
    "SD": "South Dakota", "TN": "Tennessee", "TX": "Texas", "UT": "Utah",
    "VT": "Vermont", "VA": "Virginia", "WA": "Washington", "WV": "West Virginia",
    "WI": "Wisconsin", "WY": "Wyoming"
}

# Create a new column 'statename' by mapping the abbreviations
df['statename'] = df['state'].map(state_abbreviations)

# Save the updated DataFrame to a new CSV file
df.to_csv("cleaned_and_merged_crimedata_with_statename.csv", index=False)

# Display the first few rows of the updated DataFrame
print(df.head())


             communityName state  countyCode  communityCode  population  \
0  BerkeleyHeightstownship    NJ        39.0         5320.0       11980   
1           Marpletownship    PA        45.0        47616.0       23123   
2               Tigardcity    OR         NaN            NaN       29344   
3         Gloversvillecity    NY        35.0        29443.0       16656   
4              Bemidjicity    MN         7.0         5068.0       11245   

   householdsize  racepctblack  racePctWhite  racePctAsian  racePctHisp  ...  \
0           3.10          1.37         91.78          6.50         1.88  ...   
1           2.82          0.80         95.57          3.44         0.85  ...   
2           2.43          0.74         94.33          3.43         2.35  ...   
3           2.40          1.70         97.35          0.50         0.70  ...   
4           2.76          0.53         89.16          1.17         0.52  ...   

   larcPerPop  autoTheft  autoTheftPerPop  arsons  arsonsPerPop  \
0

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Step 1: Load the cleaned and merged crime data file
data_file = "cleaned_and_merged_crimedata_with_statename.csv"  # Updated file with 'statename'
df = pd.read_csv(data_file)

# Step 2: Extract unique state names (statename) from the dataset
unique_states = df['statename'].dropna().unique()  # Ensure no null values in 'statename'

# Step 3: Start the WebDriver
driver = webdriver.Chrome()

# Step 4: Define a function to scrape the unemployment rate for a state
def get_unemployment_rate(state_name):
    url = "https://www.bls.gov/lau/lastrk19.htm"
    driver.get(url)
    time.sleep(2)  # Allow the page to load
    
    try:
        # Locate the table containing the data
        table = driver.find_element(By.ID, "lastrk19")  # Table ID from the website
        
        # Get all rows in the table
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        # Iterate through rows to find the matching state
        for row in rows:
            try:
                # Check the <th> element for the state name
                state_cell = row.find_element(By.TAG_NAME, "th")  # State name is in <th>
                if state_name.lower() == state_cell.text.strip().lower():  # Case-insensitive match
                    # The unemployment rate is in the first <td> of this row, possibly under a <span>
                    rate_cell = row.find_elements(By.TAG_NAME, "td")[0]  # First <td>
                    rate_span = rate_cell.find_element(By.TAG_NAME, "span")
                    unemployment_rate = rate_span.text.strip()
                    print(f"{state_name}: {unemployment_rate}")
                    return unemployment_rate
            except Exception:
                continue  # Skip to the next row if there is an issue with this row
        print(f"{state_name} not found in the table.")
        return None
    except Exception as e:
        print(f"Could not retrieve unemployment rate for {state_name}: {e}")
        return None

# Step 5: Scrape unemployment rates for all unique states
state_unemployment = {}
for state in unique_states:
    rate = get_unemployment_rate(state)
    state_unemployment[state] = rate

# Step 6: Convert the results to a DataFrame
unemployment_df = pd.DataFrame(list(state_unemployment.items()), columns=['state_name', 'unemployment_rate'])

# Step 7: Save the unemployment rates to a CSV file
unemployment_df.to_csv("state_unemployment_rates.csv", index=False)

# Close the WebDriver
driver.quit()

# Display the first few rows of the unemployment rates DataFrame
print(unemployment_df.head())


KeyboardInterrupt: 